# Классификация изображений. Сверточные сети. Полносвязные сети с предобучением и дообучением.

Выполнили Вилюмсон Александр, Николаев Егор из группы 4136 

In [ ]:
import os

base_dir = "archive/flowers"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")

def count_images_per_class(directory):
    counts = {}
    if os.path.exists(directory):
        for cat in os.listdir(directory):
            category_path = os.path.join(directory, cat)
            if os.path.isdir(category_path):
                counts[cat] = len([img for img in os.listdir(category_path) if img.endswith(".jpg")])
    return counts

train_counts = count_images_per_class(train_dir)
val_counts = count_images_per_class(val_dir)

# Вывод результата
print("Количество изображений в выборке 'train':")
for category, count in train_counts.items():
    print(f"  {category}: {count}")

print("\nКоличество изображений в выборке 'val':")
for category, count in val_counts.items():
    print(f"  {category}: {count}")

In [ ]:
import pandas as pd

# Путь к директориям train и val
base_dir = "archive/flowers"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")

# Функция для создания DataFrame из указанной директории
def create_dataframe(directory, type):
    data = []
    for category in os.listdir(directory):
        category_path = os.path.join(directory, category)
        if os.path.isdir(category_path):
            for img in os.listdir(category_path):
                if img.endswith(".jpg"):
                    data.append({
                        "file_path": os.path.join(category_path, img),
                        "class": category,
                        "dataset_type": type,
                    })
    return pd.DataFrame(data)

train_df = create_dataframe(train_dir, "train")
val_df = create_dataframe(val_dir, "val")
data_df = pd.concat([train_df, val_df], ignore_index=True)

In [ ]:
data_df['class'].value_counts()

In [ ]:
from PIL import Image
import numpy as np

# Задаем целевой размер
TARGET_SIZE = (224, 224)

# Функция для преобразования изображения
def preprocess_image(file_path, target_size=TARGET_SIZE):
    with Image.open(file_path) as img:
        img = img.resize(target_size)
        img = img.convert('RGB')    
        img_array = np.array(img)   
        return img_array

data_df['processed_image'] = data_df['file_path'].apply(preprocess_image)
print(data_df['processed_image'].iloc[0].shape)

In [ ]:
data_df['normalized_image'] = data_df['processed_image'].apply(lambda x: x / 255.0)
data_df.head()

In [ ]:
# Разделяем данные
train_data = data_df[data_df['dataset_type'] == 'train']
val_data = data_df[data_df['dataset_type'] == 'val']

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_data['class'])
y_val = label_encoder.transform(val_data['class'])

X_train = np.stack(train_data['normalized_image'].values)
X_val = np.stack(val_data['normalized_image'].values)

X_train.head()